In [1]:
!pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 8.8 MB/s 
     |████████████████████████████████| 462 kB 58.8 MB/s 


In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [4]:
df = pd.read_csv('gujarati-train.csv')
df.head()

,headline,Label
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports


In [5]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [6]:
count_length()

In [7]:
df.head(10)

,headline,Label,word_count
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports,10
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports,10
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports,10
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11
6,"IND vs WI: ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી...",Sports,15
7,IPL 2022: હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુ...,Sports,12
8,IPL 2022ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ ...,Sports,17
9,IPL 2022: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે...,Sports,12


In [8]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
f = open("Gujarati_stopwords.txt", "r",encoding="utf8")
from nltk.corpus import stopwords
stopwords_gu = []
for i in f:
    i=i.split("\n")
    stopwords_gu.append(str(i[0]))
stopwords_en = stopwords.words('english')
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
stopwords_lst = stopwords_gu + stopwords_en + punctuations

In [10]:
df['label_id'] = df.Label.factorize()[0]
import nltk
import re
def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text 
 

In [11]:
for i in range(len(df)):
    df['headline'][i] = processText(df['headline'][i])
count_length()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df.head(10)

,headline,Label,word_count,label_id
0,: - ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓન...,Sports,10,0
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10,0
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, ની સ્પિ...",Sports,10,0
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11,0
4,ક્યાં રમાશે ના મેચ? સૌરવ ગાંગુલીએ આપ્યો આ જવાબ,Sports,10,0
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11,0
6,": ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી પ્રથમ ...",Sports,15,0
7,": હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુશ્કેલ, ...",Sports,12,0
8,ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ વિદેશી...,Sports,17,0
9,: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે મેગા ઓ...,Sports,12,0


In [13]:
!pip install indic-nlp-library

     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 20.2 MB/s 


In [14]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens

In [15]:
df['headline'] = df.headline.apply(lambda x: tokenization(x))
df.head(10)

,headline,Label,word_count,label_id
0,"[:, -, ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલ...",Sports,10,0
1,"[રોહિત, -, કોહલીના, વિવાદ, પર, ગાવસ્કરે, કર્યો...",Sports,10,0
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, ,,...",Sports,10,0
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,11,0
4,"[ક્યાં, રમાશે, ના, મેચ, ?, સૌરવ, ગાંગુલીએ, આપ્...",Sports,10,0
5,"[ટીમ, ઈન્ડિયાનો, આ, ખેલાડી, વેડફી, રહ્યો, છે, ...",Sports,11,0
6,"[:, ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, ,, ફરી,...",Sports,15,0
7,"[:, હરાજીમાં, આ, ખેલાડીઓનું, વેચાવવું, છે, મુશ...",Sports,12,0
8,"[ના, મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, છે, આ,...",Sports,17,0
9,"[:, ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, છે...",Sports,12,0


In [16]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (stopwords_lst)]
count_length()
df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,headline,Label,word_count,label_id
0,"[ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલાડીઓની...",Sports,6,0
1,"[રોહિત, કોહલીના, વિવાદ, ગાવસ્કરે, કર્યો, મોટો,...",Sports,10,0
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, સ્...",Sports,7,0
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,10,0
4,"[ક્યાં, રમાશે, મેચ, સૌરવ, ગાંગુલીએ, જવાબ]",Sports,6,0
5,"[ટીમ, ઈન્ડિયાનો, ખેલાડી, વેડફી, રહ્યો, ટેલેન્ટ...",Sports,9,0
6,"[ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, મેચ, નહી, ...",Sports,10,0
7,"[હરાજીમાં, ખેલાડીઓનું, વેચાવવું, મુશ્કેલ, કરોડ...",Sports,7,0
8,"[મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, વિદેશી, ખે...",Sports,10,0
9,"[ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, મેગા,...",Sports,8,0


In [17]:
df['headline'] = df['headline'].str.join(" ")
df.head(10)

,headline,Label,word_count,label_id
0,ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓની બોલી,Sports,6,0
1,રોહિત કોહલીના વિવાદ ગાવસ્કરે કર્યો મોટો ખુલાસો...,Sports,10,0
2,પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ સ્પિડથી ...,Sports,7,0
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,10,0
4,ક્યાં રમાશે મેચ સૌરવ ગાંગુલીએ જવાબ,Sports,6,0
5,ટીમ ઈન્ડિયાનો ખેલાડી વેડફી રહ્યો ટેલેન્ટ ગાવસ્...,Sports,9,0
6,ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ મેચ નહી રમી શકે...,Sports,10,0
7,હરાજીમાં ખેલાડીઓનું વેચાવવું મુશ્કેલ કરોડોમાં ...,Sports,7,0
8,મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે વિદેશી ખેલાડી કર...,Sports,10,0
9,ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં મેગા ઓક્શનમાં,Sports,8,0


In [18]:
num_classes = len(df["label_id"].value_counts())

In [19]:
y = tf.keras.utils.to_categorical(df["label_id"].values, num_classes=num_classes)

X_train, X_test, Y_train, Y_test = train_test_split(df['headline'], y, test_size=0.25, random_state=42)

In [20]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
bert_encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")

In [21]:
i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
x = bert_preprocess(i)
x = bert_encoder(x)
x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
x = tf.keras.layers.Dense(num_classes, activation='softmax', name="output")(x)

model = tf.keras.Model(i, x)

In [22]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.fit(X_train, Y_train, epochs=5)

Epoch 1/5
886/886 [==============================] - 301s 320ms/step - loss: 0.5774 - accuracy: 0.8158
Epoch 2/5
886/886 [==============================] - 289s 326ms/step - loss: 0.3729 - accuracy: 0.8724
Epoch 3/5
886/886 [==============================] - 289s 326ms/step - loss: 0.3534 - accuracy: 0.8780
Epoch 4/5
886/886 [==============================] - 288s 326ms/step - loss: 0.3469 - accuracy: 0.8789
Epoch 5/5
886/886 [==============================] - 288s 326ms/step - loss: 0.3392 - accuracy: 0.8794


In [24]:
accr = model.evaluate(X_test, Y_test)

296/296 [==============================] - 97s 325ms/step - loss: 0.3163 - accuracy: 0.8873


In [25]:
print('Accuracy: {:0.2f}'.format(accr[1]))

Accuracy: 0.89


In [26]:
model.save('/content/gdrive/My Drive/GU_BERT.h5')

In [27]:
model.save('GU_BERT.h5')

In [28]:
test_classes = []
for i in range(len(Y_test)):
    for j in range(len(Y_test[i])):
        if Y_test[i][j] == 1:
            if j==0:
                test_classes.append(0)
            if j==1:
                test_classes.append(1)
            if j==2:
                test_classes.append(2)
            if j==3:
                test_classes.append(3)
            if j==4:
                test_classes.append(4)

In [29]:
predicted_categories = []
preds = model.predict(X_test)
for i in preds:
    pred_classes = np.argsort(i)[-1:][::-1]
    predicted_categories.append(pred_classes[0])

In [30]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
report = classification_report(test_classes, predicted_categories)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.89      0.90      1982
           1       0.91      0.83      0.87      1818
           2       0.82      0.91      0.86      1828
           3       0.90      0.90      0.90      1916
           4       0.92      0.91      0.91      1901

    accuracy                           0.89      9445
   macro avg       0.89      0.89      0.89      9445
weighted avg       0.89      0.89      0.89      9445



In [31]:
mat = confusion_matrix(test_classes, predicted_categories)
mat

array([[1761,   21,   32,  101,   67],
       [  21, 1511,  231,   23,   32],
       [  25,   89, 1659,   28,   27],
       [  83,   21,   53, 1729,   30],
       [  63,   27,   51,   39, 1721]])

In [32]:
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

In [33]:
sample_text1 = ["શેરમાર્કેટમાં કડાકો:10 દિવસમાં Mcap 15 લાખ કરોડ ઘટ્યું; સેન્સેક્સ 678 પોઇન્ટ તૂટ્યો, ઇન્વેસ્ટર્સે 3 દિવસમાં 6.15 લાખ ગુમાવ્યાં"]
preds = model.predict(sample_text1)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

business 99.93 %


In [34]:
sample_text2 = ["SRKના દીકરાની ઘરવાપસી LIVE:આર્યન ખાનને 'મન્નત'થી આર્થર રોડ જેલનું 13 કિલોમીટરનું અંતર કાપતાં 28 દિવસ થયા"]
preds = model.predict(sample_text2)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

entertainment 97.69 %


In [35]:
sample_text3 = ["લોન્ચ થયો Oppo નો Waterproof સ્માર્ટફોન, ધાંસૂ કેમેરા સાથે મળશે ગજબના ફીચર્સ"]
preds = model.predict(sample_text3)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

tech 99.83 %


In [36]:
sample_text4 = ["ઉત્તરાખંડમાં ભાજપને મોટો ઝટકો, કેબિનેટ મંત્રી પદેથી રાજીનામું આપીને આ નેતા કોંગ્રેસમાં થયા સામેલસ"]
preds = model.predict(sample_text4)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

politics 67.83 %


In [37]:
sample_text5 = ["સાઉથ આફ્રિકન સ્પિનર શેન વ્હાઈટહેડે ઈનિંગમાં ૧૦ વિકેટ ઝડપી ઈતિહાસ રચ્યો"]
preds = model.predict(sample_text5)[0]
pred_classes = np.argsort(preds)[-1:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

for c, p in zip(classes, props):
    print("{} {:.2f} %".format(c,p*100))

sports 99.48 %
